# 百度网盘AI大赛-文档图片超分比赛第1名方案


# 一、赛题分析
此次大赛主题结合日常生活常见情景展开，大家平时在使用手机进行拍照、扫描的时候，往往会需要将图片放大的场景，可是分辨率放大，图片中的元素都变得模糊起来，进而难以使用。本次比赛希望选手们通过算法等知识技术，帮助人们将因为放大而模糊的图片复原，提高图片分辨率，实现文档图像的“无损放大”，让其重新发挥作用。



# 二、 数据分析
- 本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共3000个样本，A榜测试集共200个样本，B榜测试集共200个样本,抽取一部分数据如图：：
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.png)

-

# 二、评价标准
评价指标为 PSNR 和 MSSSIM；

用于评价的机器环境仅提供两种框架模型运行环境：paddlepaddle 和 onnxruntime，其他框架模型可转换为
上述两种框架的模型；

机器配置：V100，显存32G，内存30G；

单张图片耗时>5s，决赛中的性能分数记0分。

由评价标准可知，不能使用大模型。

# 三、模型设计

- 针对图像超分这个任务，我们查阅了相关资料，传统的文本评价标准是以OCR模型识别的准确率来比较的，目的是提高文字识别准确率，但是此次百度网盘的文档超分比赛，评分的标准是PSNR与MS_SSIM，所以该任务属于重建优化的范畴，所以我们选择了基于卷积架构的rcan作为我们此次的baseline,相比于transformer架构的模型，卷积架构会更加稳定。本次比赛要产生原图放大2倍和放大4倍的结果，如果分开计算必然耗时严重，所以我们将两个任务合并。

![](![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/pipeline.png)

- 从网络结构图上可以直观的看出改进后rcan由单分支网络变成了双分支网络，一个分支负责产生放大2倍的结果，一个分支负责产生放大4倍的结果。原始的racn使用了均值漂移的聚类算法，考虑到本次训练数据集的像素值大部分只分布在0和255附近，便删去这一部分。

- 训练数据集的内容分为中文图片和英文图片，考虑到语言之间的差异性以及语言文字的先验性，决定中英文分开训练，所以在测试的时候需要一个二分类模型来判断中文还是英文，我们使用精剪后的vgg来做二分类。

![](![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/pipeline.png)

- 由于分类任务非常容易，所以该网络只由几层卷积构成。



# 四、数据处理与增强

### 数据划分
- 训练的文档的图像尺寸很大，而且四周存在很多纯色空白，为减小训练过程中的IO占用以及空白样本过于简单不利于模型收敛，应该将其裁剪掉四周的空白区域，在进行切块处理
- 以步长64，大小为128进行切块

### 数据筛选
- 在进行切块处理后，仍然存在一些图像块是白色的，由于他们很容易恢复，对模型的收敛可能会产生负面的影响，导致收敛不充分
- 我们通过计算图像梯度，并将图像梯度小于5的图像块删掉

### 数据增广
- 为保留文字信息的先验性，不进行任何数据增强，只做归一化处理

# 五、训练细节
- 训练配置
总迭代数：450000 iteration
我们采用batch size为16来进行训练450000次迭代。
我们采用了余弦退火的学习率策略来优化网络，学习率我们采用1e-4，优化器为Adam。
- 损失函数为L1Loss

# 六、测试细节
- 测试图片预先全部输入到二分类模型中，得到包含中文图片路径的list和包含英文图片路径的list
- 中文图片路径的list送入到中文模型中，英文图片路径的list送入到英文模型中
- 在测试中我们输入到模型中的尺寸为256X256,由于我们将原图裁剪，所以图片边缘的处理尤为重要，我们采用overlap的方法，使图片patches之间的重叠部分为24，这就保证了在图片patcher复位的过程中前一个patch取到重叠区域的一半，也就是12个像素长度，后一个patch从重叠区域的一半开始取，这就保证了patches边缘在复位的时候可以很好的过渡。
- 整个测试过程为串行


# 七、代码结构
### chinese_code && english_code:
- dataloader: 定义数据增强函数和数据集
- models: 定义网络模型
- utils: 定义评价指标函数和其它函数
- checkpoint: 模型训练输出文件夹
- result:模型测试输出文件夹
- train.py: 训练脚本
- *train.txt:预先生成corp的位置信息，这个位置信息通过切分并剔除空白区域后得到的，没有直接生成裁剪好的图片是为了节约内存


# 八、上分策略

上分策略主要集中在数据清洗，中英文分开

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 代码启动过程
## erasenet训练过程

In [1]:
### 解压数据
!cd ./ && unzip -q *.zip

^C


解压完手动把所有的 x,x2,x4的文件里的文件合并，最后得到 x,x2,x4 

In [ ]:
###裁剪四周空白区域
import os
import cv2
import numpy as np

from pdb import set_trace as stx

def trim(img):
        val=230
        sp = img.shape  # 获取图像形状：返回【行数值，列数值】列表,sp3为RGB
        sz1 = sp[0]
        sz2 = sp[1]
        n1 = 0
        n2 = sz1
        n3 = 0
        n4 = sz2    #n1~n4代表上下左右的分界线
        for i in range(sz1):
            for j in range(sz2):
                px = np.mean(img[i, j])
                if px < val:   #我的图片是灰度图像，如果是彩色的图，可以用255*3以下的值来判定范围
                    n1 = i
                    break
            else:
                continue
            break
        for m in range(sz1 - 1, 0, -1):
            for n in range(sz2):
                px = np.mean(img[m, n])
                if px < val:
                    n2 = m
                    break
            else:
                continue
            break
        for p in range(sz2):
            for q in range(sz1):
                px = np.mean(img[q, p])
                if px < val:
                    n3 = p
                    break
            else:
                continue
            break
        for o in range(sz2 - 1, 0, -1):
            for u in range(sz1):
                px = np.mean(img[u, o])
                if px < val:
                    n4 = o
                    break
            else:
                continue
            break
      
        return max(0,n1-5),min(sz1,n2+5),max(0,n3-5),min(sz2,n4+5)


if __name__=="__main__":
    #原图路径
    x_path='./x/'
    x2_path='./x2/'
    x4_path='./x4/'
    #保存路径
    nn_x_path='./xx/'
    nn_x2_path='./xx2/'
    nn_x4_path='/xx4/'

    file_names=os.listdir(x_path)
    count=0
    for name in file_names:
        print(count)
        x_img_path=os.path.join(x_path,name)
        x2_img_path=os.path.join(x2_path,name)
        x4_img_path=os.path.join(x4_path,name)

        x_img=cv2.imread(x_img_path)
        x2_img=cv2.imread(x2_img_path)
        x4_img=cv2.imread(x4_img_path)
        y1,y2,x1,x2=trim(x_img)
        x_img=x_img[y1:y2,x1:x2,:]
        x2_img=x2_img[y1*2:y2*2,x1*2:x2*2,:]
        x4_img=x4_img[y1*4:y2*4,x1*4:x2*4,:]

        cv2.imwrite(x_path+name,x_img)
        cv2.imwrite(x2_path+name,x2_img)
        cv2.imwrite(x4_path+name,x4_img)
        count+=1


从 xx 文件夹里挑出所有的中文图片，xx2,xx4中对应的文件也剪切出来，得到chinese_x,chinese_x2,chinese_x4
同理剩下的英文图片组成 english_x,english_x2,english_x4

In [ ]:
####生成训练使用的包含corp位置的train.txt,只需要对chinese_x,和english_x生成corp位置，*_x2,*_x4对应的corp坐标分别乘以2和4就是对应label
import os
import cv2
import numpy as np

x_path='./chinese_x' 
file_names=os.listdir(x_path)
base=128
retreat=64

with open ("c_train.txt",mode="a") as f:
    for name in file_names:
        x_img_path=os.path.join(x_path,name)
        x_img=cv2.imread(x_img_path)
        w_split=[]
        h_split=[]
        h,w,c=x_img.shape
        dynamic_w=0
        dynamic_h=0
        while(dynamic_w<w):
            w_split.append(dynamic_w)
            dynamic_w+=base-retreat
            if(dynamic_w+base>=w):
                dynamic_w=w-base
                w_split.append(dynamic_w)
                break

        while(dynamic_h<h):
            h_split.append(dynamic_h)
            dynamic_h+=base-retreat

            if(dynamic_h+base>=h):
                dynamic_h=h-base
                h_split.append(dynamic_h)
                break
    
        boundbox=[]
        for i in range(len(w_split)):
            for j in range(len(h_split)):
                if(i==0 and j ==0 ):
                    boundbox.append([base*i,base,base*j,base])
                elif(i==0):
                    boundbox.append([base*i,base,h_split[j],h_split[j]+base])
                elif(j==0):
                    boundbox.append([w_split[i],w_split[i]+base,base*j,base])
                else:
                    boundbox.append([w_split[i],w_split[i]+base,h_split[j],h_split[j]+base])
        boundbox=np.array(boundbox)
    
        for i in range(len(boundbox)):
            box=boundbox[i]
            cur=x_img[box[2]:box[3],box[0]:box[1],:]
            sobelx = cv2.Sobel(cur,cv2.CV_64F,1,0,ksize=3)
            sobelx = cv2.convertScaleAbs(sobelx)
            sobely = cv2.Sobel(cur,cv2.CV_64F,0,1,ksize=3)
            sobely = cv2.convertScaleAbs(sobely)
            sobelxy = cv2.addWeighted(sobelx,0.5,sobely,0.5,0)
            val=np.mean(sobelx)
            if(val<4):
                continue
            f.write(name+' '+str(box[2])+','+str(box[3])+','+str(box[0])+','+str(box[1]))
            f.write("\n")
    f.close()

In [1]:
## 安装训练所需的python包
! pip install torch==1.8.1
! pip install torchvision==0.9.1
! pip install natsort  
! pip install opencv-python
! pip install scikit-image
! pip install warmup_scheduler
! pip install pillow

^C
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2998 sha256=e51155ddd5e11b5b054fc3308df854a41a5ec4e7f4f81ba9f62f2fe38d60363b
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\af\09\35\3ef9059c328587fc37578f2e6b4039b3fbe6485a99ae9fc41e
Successfully built warmup-scheduler
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
## 利用L1Loss训练
! cd chinese_model && python train.py 
! cd english_model && python train.py

Loading weights into state dict...


Traceback (most recent call last):
  File "train.py", line 24, in <module>
    yolo = YOLO()
  File "c:\Users\Administrator\Desktop\BaiduDiskAI_DocRemoveCover_top3\erasenet_code\models\yolo.py", line 48, in __init__
    self.generate()
  File "c:\Users\Administrator\Desktop\BaiduDiskAI_DocRemoveCover_top3\erasenet_code\models\yolo.py", line 77, in generate
    self.net = self.net.cuda()
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 491, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 387, in _apply
    module._apply(fn)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 387, in _apply
    module._apply(fn)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 409, in _apply
    param_applied = fn(param)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 491,

In [ ]:
## onnxruntime-gpu 测试脚本 ##
    
! cd test_code/ && python predict.py {your_test_data_path} {x2_save_path} {x4_save_path}

: 